## 데이터세트 분리

In [2]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [3]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y
    
    def __len__(self):
        return self.length

In [4]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [5]:
dataset = CustomDataset('./datasets/non_linear.csv')
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])
print(f'Training Data Size : {len(train_dataset)}')
print(f'Validation Data Size : {len(validation_dataset)}')
print(f'Testing Data Size : {len(test_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

Training Data Size : 160
Validation Data Size : 20
Testing Data Size : 20


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)

lr = 0.0001

optimizer = optim.SGD(model.parameters(), lr=lr)

In [7]:
epochs = 10000

for epoch in range(epochs):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}')

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.0982, -1.7027]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5389], device='cuda:0', requires_grad=True)], Cost : 0.081
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.0989, -1.7027]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.5068], device='cuda:0', requires_grad=True)], Cost : 0.080
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1006, -1.7028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4933], device='cuda:0', requires_grad=True)], Cost : 0.082
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1006, -1.7029]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4876], device='cuda:0', requires_grad=True)], Cost : 0.081
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1009, -1.7028]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.4852], device='cuda:0', requires_grad=True)]

In [8]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        print(f'X : {x}')
        print(f'Y : {y}')
        print(f'Outputs : {outputs}')
        print('--------------------')

X : tensor([[33.6400,  5.8000],
        [ 0.2500, -0.5000],
        [50.4100,  7.1000],
        [ 2.5600,  1.6000]], device='cuda:0')
Y : tensor([[ 95.0500],
        [  2.4800],
        [144.2400],
        [  6.0700]], device='cuda:0')
Outputs : tensor([[ 94.9377],
        [  2.1100],
        [144.7343],
        [  5.6982]], device='cuda:0')
--------------------
X : tensor([[ 0.1600, -0.4000],
        [16.8100, -4.1000],
        [14.4400,  3.8000],
        [ 5.7600,  2.4000]], device='cuda:0')
Y : tensor([[ 1.5500],
        [59.1100],
        [38.7700],
        [13.8100]], device='cuda:0')
Outputs : tensor([[ 1.6606],
        [59.5993],
        [38.7966],
        [14.2604]], device='cuda:0')
--------------------
X : tensor([[44.8900,  6.7000],
        [38.4400, -6.2000],
        [15.2100,  3.9000],
        [46.2400, -6.8000]], device='cuda:0')
Y : tensor([[127.8900],
        [130.4100],
        [ 41.1700],
        [155.4300]], device='cuda:0')
Outputs : tensor([[128.2958],
        [130